In [4]:
from ipywidgets import Image
import ipywidgets as widgets
from ipycanvas import Canvas, RoughCanvas, hold_canvas
import sys
# %load_ext autoreload
# %autoreload 2
sys.path.append("..")

In [5]:
import Optimizers
from NeuralNetwork import NeuralNetwork
from Layer import Layer
from ActivationFunction import ActivationFunction
import numpy as np
neuralNetwork = NeuralNetwork(
    layers=[
        Layer(neurons=28*28, activation=ActivationFunction.identity),
        Layer(neurons=100, activation=ActivationFunction.leaky_reLU),
        Layer(neurons=50, activation=ActivationFunction.leaky_reLU),
        Layer(neurons=10, activation=ActivationFunction.softmax),
    ],
    optimizer=Optimizers.GradientDescent(learning_rate=0.05),
    # l2_regularization=0.0001
)
neuralNetwork.load_from_file("trained_models/epoch_10_layer_100_50_leakyReLU")

In [10]:
from ipycanvas import Canvas, RoughCanvas, hold_canvas
canvas = Canvas(width=280, height=280, sync_image_data=True, layout=widgets.Layout(border='2px solid #1A2028'))

position = None
shape = []

brush = Image.from_file('brushes/brush.png')

clear_canvas_btn = widgets.Button(
    description='Reset',
    button_style='',  # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Clear canvas',
    icon='refresh',  # (FontAwesome names without the `fa-` prefix)
    layout=widgets.Layout(margin='15px 0 0')
)

text_value = widgets.Textarea(
    value='',
    placeholder='Type something',
    description=''
)

progress_bar_generator = lambda number : widgets.FloatProgress(
    value=0,
    min=0,
    max=1,
    description=f'{number}',
    bar_style='warning',
    style={'bar_color': 'tomato', 'label_min_width': "10px"},
    orientation='horizontal',
)
progress_bars = list((progress_bar_generator(i) for i in range(10)))
progress_bars_widget = widgets.VBox(progress_bars)


def on_mouse_down(x, y):
    global drawing
    global position
    global shape

    drawing = True
    position = (x, y)
    shape = [position]


def on_mouse_move(x, y):
    global drawing
    global position
    global shape

    if not drawing:
        return

    #with hold_canvas(canvas):
    canvas.draw_image(brush, x - 14, y - 14)

    position = (x, y)

        # arr1 = canvas.get_image_data()
        # canvas.to_file('my_file.png')

    shape.append(position)


def on_mouse_up(x, y):
    global drawing
    global position
    global shape

    drawing = False

    #with hold_canvas(canvas):
    canvas.draw_image(brush, x - 14, y - 14)
        # canvas.fill_polygon(shape)
        # arr1 = canvas.get_image_data()
        # text_value.value = str(canvas.get_image_data())


    shape = []

canvas.on_mouse_down(on_mouse_down)
canvas.on_mouse_move(on_mouse_move)
canvas.on_mouse_up(on_mouse_up)

canvas.stroke_style = '#000'
canvas.line_width = 5
canvas.laine_cape = "butt"


def get_array(*args, **kwargs):
    # Do something with arr
    # arr = canvas.get_image_data()
    # arr = arr[...,-1] / 255
    # text_value.value = str(arr)
    
    arr = canvas.get_image_data()
    if(np.all(arr==0)):
        for i, progress_bar in enumerate(progress_bars):
            progress_bar.value = 0
        return
    arr = arr[...,-1]
    smaller_img = arr[::10, ::10] / 255
    out = neuralNetwork.forward_propagate(smaller_img.reshape(28*28))
    #text_value.value = str(np.around(out, decimals=2))
    for i, progress_bar in enumerate(progress_bars):
        progress_bar.value = out[i]
    


# Listen to changes on the ``image_data`` trait and call ``get_array`` when it changes.
canvas.observe(get_array, 'image_data')


def on_clear_canvas_btn_clicked(b):
    canvas.clear()
    # canvas.rotate(45)

clear_canvas_btn.on_click(on_clear_canvas_btn_clicked)
centered_layout = widgets.Layout(align_items='center')
#widgets.VBox((canvas), layout=widgets.Layout(border='1px solid #ccc'))
widgets.HBox(
    (    
         widgets.VBox((canvas, clear_canvas_btn),layout=centered_layout),
         progress_bars_widget
    ),
    layout=centered_layout
)
